<a href="https://colab.research.google.com/github/amindadgar/Evolutionary-Computing/blob/colab/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Evolutionary Computing Exercise No. 3
**Stu. Name:** Mohammad Amin Dadgar

**Stu. Id:** 4003624016

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
RESULTS_DIR = '/content/gdrive/MyDrive/EC Project/'

In [ ]:
## comment if starting from scratch
## or
with open(RESULTS_DIR + 'initial_population.txt', 'r') as file:
  information = file.read()
  print(information)

3716117525138111365475161721741: 0.6662291765213013
2309043794989104922761629065325: 0.6473543047904968
9721641961530748913170990344988: 0.693155312538147
3619345711440214088615985959855: 0.6342633187770843
1737391375797669820948839413817: 0.6932214498519897
8980170163293842844978931065018: 0.6931642532348633
8804142551804826360071476093978: 0.8433380603790284
8425914106406874751955254232596: 0.693167245388031
4840888250790645929924090931315: 0.7270308494567871
4166078485576676288360495947997: 0.47462679743766784


In [ ]:
population = []
population_fitness = []
for individual_info in information.split('\n'):
  individual, individual_fitness = individual_info.replace(' ', '').split(':') 
  ## convert to their specific types
  individual = str(individual)
  individual_fitness = float(individual_fitness)

  population.append(individual)
  population_fitness.append(individual_fitness)

## Algorithm Configurations

### Representation
Representation for our algorithm has three levels.

- First level is just showing a concept for the network architecture in which the green boxes are optional (can be shown or not, 0 or 1)

    <img src='architecture_level1.png'>
- Second level is representing the transformer layer

    <img src='architecture_level2.png'>
- And the third level is showing the FFN and feed-forward network architectures

    <img src='architecture_level3.png'>

For each gene we can use numerical values which are 0 to 9. As we can see in the exercise each hyperparameter has 2, 3, or 4 values and by that we can convert the numerical values from intervals as below

**Two valued hyperparameters:**
- 0:4 → 0
- 5:9 → 1

**Three valued hyperparameters:**
- 0:3 → 0
- 4:6 → 1
- 7:9 → 2

**Four valued hyperparameters:**
- 0:1 → 0
- 2:4 → 1
- 5:7 → 2
- 8:9 → 3

**100 valued hyperparameters:**
- 0 → 0
- 1 → 10
- 2 → 20
- ...
- 9 → 90
- And 100 would not really work in our 100 valued parametered dropout probability, so we'll exclude it from possible values. 


So to represent a chromsome `1+3×(3+3×2)+3` bits are requierd. The first `1` is $d_{model}$, then in the clause `3×(3+3×2)+3`, the first `3` is showing three possible transform layers. In the paranthesis the first `3` is showing the bit for attention head count and normalization layers then the `3` in multiplication is the count of possible hyperparameters in FFN layer (we could have 2 FFN layer in a transformer) and the last `3` in the equation is the final FFN layer for the network. So `31` bits will be used.

### Recombination and Mutation
To combine chromsomes for two recombination and mutation methods, we should assume the three level architecture for it. To that aim, We had implemented the mutation based on the conceptual (level 1) chromosome meaning the transformers and FFN layer are mutated as a pack. For recombination method, a normal single point or uniform cross-over can be used.

### End Condition
The end condition is the count of generations, which is 10 as given in the exercise.
### Fitness Function
The fitness function is assumed the training 5 epoch of the transformer network and returning the 5 average test accuracy of it. As we will see the transfomer network does have high computational complexity (in time and hardware resources), so that's the reason that the end condition, averaging count, and epochs are set as low as they can be. 

## Evolutionary Algorithm Part

In [4]:
from population import generate_population
from combination import mutation_creep, single_point
from util import convert_genotype_to_phenotype_values, map_hyperparameters
from fitness import static_fitness
from selection import binary_tournament
from transformer_network_creator import fitness_evaluate
import numpy as np

In [5]:
def algorithm_run(pop_count, SELECTION_METHOD, FITNESS_FUNCTION, MUTATION_METHOD, RECOMBINATION_METHOD, start_generation, last_population=None, last_population_fitness = None, p_m=0.1, p_c =0.9 ,max_generations = 10, RESULTS_DIR='/content/gdrive/MyDrive/EC Project/'):
    """
    one constraint should be always given as input, the maximum capacity or maximum distance
    """
    ## if we had generated population before
    if last_population is None:
      population = generate_population(pop_size=pop_count)
      fitness_pop = []
      for chromosome in population:
          chromosome_fitness = FITNESS_FUNCTION(chromosome, None, 5)
          fitness_pop.append(chromosome_fitness)
    else:
      print('Population is loaded from file!\n')
      fitness_pop = last_population_fitness
      population = last_population


    best_chromosome = None
    best_chromsome_fitness = None
    
    generation_num = 0
    if start_generation is not None:
      generation_num = start_generation
    for generation_idx in range( generation_num ,max_generations):
        print('*' * 30 + f'Generation Number: {generation_idx}' + '*' * 30)


        ## create pair of the parents
        parent_pairs = []
        for _ in range(pop_count):
            pair = SELECTION_METHOD(population, fitness_pop)
            parent_pairs.append(pair)

        
        offsprings = []
        fitness_offsprings = []
        for parents in parent_pairs:
            recombination_p = np.random.random()

            ## the offspring for this iteration
            ## first save the parents to change them later
            iteration_offspring = [parents[0], parents[1]]
            
            ######## Recombination ########
            if recombination_p < p_c:
                offspring1, offspring2 =  RECOMBINATION_METHOD(iteration_offspring[0], iteration_offspring[1])

                iteration_offspring = [offspring1, offspring2]

            ######## Mutation ########
            offspring1 = MUTATION_METHOD(iteration_offspring[0], p_m)
            offspring2 = MUTATION_METHOD(iteration_offspring[1], p_m)

            iteration_offspring = [offspring1, offspring2]
                
            ## finally append the genarated offsprings to offspring array 
            offsprings.append(iteration_offspring[0])
            offsprings.append(iteration_offspring[1])
            
            fitness_offsprings.append(FITNESS_FUNCTION(iteration_offspring[0], RESULTS_DIR + f'generation_number_{generation_idx}.txt', 5))
            fitness_offsprings.append(FITNESS_FUNCTION(iteration_offspring[1], RESULTS_DIR + f'generation_number_{generation_idx}.txt', 5))
            
                
        ######## Replacement ########

        ## the whole generation: parents + offsprings
        generation_population = population.copy()
        generation_population.extend(offsprings)

        ## whole generation fitness: parents fitness + offsprings fitness
        generation_fitness = fitness_pop.copy()
        generation_fitness.extend(fitness_offsprings)

        ## the sorted generation
        generation_population_sorted = np.array(generation_population)[np.argsort(generation_fitness)[::-1]]
        generation_fitness_sorted = np.sort(generation_fitness)

        ## Step 10
        ## extract the best of the new generation
        best_of_generation_population = generation_population_sorted[:pop_count]
        best_of_generation_fitness = generation_fitness_sorted[:pop_count]

        best_chromosome = generation_population_sorted[0]
        best_chromsome_fitness = generation_fitness_sorted[0]
        
        ## save them into the original population arrays
        population = best_of_generation_population.tolist()
        fitness_pop = best_of_generation_fitness.tolist()

    return best_chromosome, best_chromsome_fitness

## Start

In [6]:
## Reading the last saved population
## comment this code if you are starting from scratch
with open(RESULTS_DIR + 'generation_number_1.txt', 'r') as file:
  information = file.read()
  print(information)

population = []
population_fitness = []
for individual_info in information.split('\n'):
  if individual_info != '':
    individual, individual_fitness_str_arr = individual_info.replace(' ', '').split(':') 
    ## convert to their specific types
    individual = str(individual)
  
    float_data_arr = individual_fitness_str_arr.replace('[', '').replace(']', '').split(',')
    float_data_arr = [float(data) for data in float_data_arr]
    individual_fitness = np.mean(float_data_arr)

    population.append(individual)
    population_fitness.append(individual_fitness)


2309043794000000029924090931000: [0.6663390398025513, 0.6699479818344116, 0.7486469745635986, 0.7779876589775085, 0.6729717254638672]
4734137673790645900924090931315: [0.6992217302322388, 0.7173842191696167, 0.6931620240211487, 0.6931490898132324, 0.7502572536468506]
3842168202790645929924090931000: [0.6881181001663208, 0.7192521095275879, 0.684798538684845, 0.4534873366355896, 0.6519611477851868]
4846117525138111365475161721741: [0.5240600109100342, 0.6059545278549194, 0.6888200044631958, 0.6664575934410095, 0.5362326502799988]
3619345711404826360071476093978: [0.693255603313446, 0.47140073776245117, 0.4153226315975189, 0.46501195430755615, 0.5552911162376404]
4840888250840214088615985959855: [0.6926466226577759, 0.6931487321853638, 0.6931583881378174, 0.35912811756134033, 0.6931068897247314]
2300888250000000000000000000596: [0.4447859227657318, 0.5386607646942139, 0.5133048295974731, 0.4687635898590088, 0.4636898338794708]
4849043794000000000924090931315: [0.6484853029251099, 0.8324

In [ ]:
## starting the algorithm with a static fitness value
## the algorithm will run randomly, but we want to debug any problems if it has
answer_chromosome, answer_chromsome_fitness = algorithm_run(pop_count=10, 
                SELECTION_METHOD=binary_tournament, 
                FITNESS_FUNCTION=fitness_evaluate, 
                MUTATION_METHOD=mutation_creep, 
                RECOMBINATION_METHOD=single_point,
                p_m=0.1,
                p_c=0.9,
                max_generations=10, 
                last_population=population,
                last_population_fitness=population_fitness, 
                start_generation=2)

Population is loaded from file!

******************************Generation Number: 2******************************
17464789/17464789 [==============================] - 2s 0us/step
25000 Training sequences
25000 Test sequences
Epoch 1/5
391/391 - 18s - loss: 0.6936 - accuracy: 0.5012 - 18s/epoch - 46ms/step
Epoch 2/5
391/391 - 13s - loss: 0.6931 - accuracy: 0.5030 - 13s/epoch - 33ms/step
Epoch 3/5
391/391 - 13s - loss: 0.6932 - accuracy: 0.5002 - 13s/epoch - 33ms/step
Epoch 4/5
391/391 - 13s - loss: 0.6932 - accuracy: 0.4969 - 13s/epoch - 33ms/step
Epoch 5/5
391/391 - 13s - loss: 0.6932 - accuracy: 0.4974 - 13s/epoch - 33ms/step
782/782 [==============================] - 10s 12ms/step - loss: 0.7026 - accuracy: 0.5000
25000 Training sequences
25000 Test sequences
Epoch 1/5
391/391 - 16s - loss: 0.6935 - accuracy: 0.4980 - 16s/epoch - 40ms/step
Epoch 2/5
391/391 - 13s - loss: 0.6932 - accuracy: 0.4962 - 13s/epoch - 33ms/step
Epoch 3/5
391/391 - 13s - loss: 0.6932 - accuracy: 0.4994 - 13s/

(128,
 ((10, 'S', 0.1, False), (30, 'R', 0.9, True), 4),
 ((20, 'S', 0.9, True), (10, 'S', 0.6, False), 8),
 ((10, 'R', 0.5, False), (5, 'R', 0.7, True), 4),
 (20, 'R', 0.6))